In [1]:
# load data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime

train = pd.read_csv('ccf_offline_stage1_train.csv')
test = pd.read_csv('ccf_offline_stage1_test_revised.csv')

submission = test[['User_id','Coupon_id','Date_received']]
train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
User_id          int64
Merchant_id      int64
Coupon_id        float64
Discount_rate    object
Distance         float64
Date_received    float64
Date             float64
dtypes: float64(4), int64(2), object(1)
memory usage: 93.7+ MB


In [3]:
# train_user_set = set(train['User_id'])
# test_user_set = set(test['User_id'])

# train_merchant_set = set(train['Merchant_id'])
# test_merchant_set = set(test['Merchant_id'])

# new_user = test_user_set - train_user_set
# new_merchant = test_merchant_set - train_merchant_set

In [4]:
def count_days(d1,d2):
    if np.isnan(d1):
        return 0
    elif np.isnan(d2):
        return 0
    else:
        d1=str(int(d1))
        d2=str(int(d2))
        date1=datetime.datetime(int(d1[0:4]),int(d1[4:6]),int(d1[6:8]))
        date2=datetime.datetime(int(d2[0:4]),int(d2[4:6]),int(d2[6:8]))
        num=(date1-date2).days
        if num<=15:
            return 1
        else:
            return 0

train['Used']=train.apply(lambda x: count_days(x.Date, x.Date_received), axis = 1)

In [5]:
train['User_Merchant'] = list(zip(train['User_id'], train['Merchant_id']))
test['User_Merchant'] = list(zip(test['User_id'], test['Merchant_id']))

In [6]:
df_used = train[train['Date_received'].notnull() & train['Date'].notnull()]
df_cons = train[train['Date_received'].isnull() & train['Date'].notnull()]
df_coupon = train[train['Date_received'].notnull() & train['Date'].isnull()]

d_user_used = dict(df_used.User_id.value_counts())
d_user_cons = dict(df_cons.User_id.value_counts())
d_user_coupon = dict(df_coupon.User_id.value_counts())

d_merchant_used = dict(df_used.Merchant_id.value_counts())
d_merchant_cons = dict(df_cons.Merchant_id.value_counts())
d_merchant_coupon = dict(df_coupon.Merchant_id.value_counts())

d_user_merchant_used = dict(df_used.User_Merchant.value_counts())
d_user_merchant_cons = dict(df_cons.User_Merchant.value_counts())
d_user_merchant_coupon = dict(df_coupon.User_Merchant.value_counts())

In [7]:
train = train[train['Date_received'].notnull()]
all_data = pd.concat([train, test], ignore_index=True, sort=False)

In [8]:
all_data['User_Used'] = all_data.User_id.apply(lambda x:d_user_used[x] if x in d_user_used else 1)
all_data['User_Cons'] = all_data.User_id.apply(lambda x:d_user_cons[x] if x in d_user_cons else 1)
all_data['User_Coupon'] = all_data.User_id.apply(lambda x:d_user_coupon[x] if x in d_user_coupon else 1)

all_data['Merchant_Used'] = all_data.Merchant_id.apply(lambda x:d_merchant_used[x] if x in d_merchant_used else 1)
all_data['Merchant_Cons'] = all_data.Merchant_id.apply(lambda x:d_merchant_cons[x] if x in d_merchant_cons else 1)
all_data['Merchant_Coupon'] = all_data.Merchant_id.apply(lambda x:d_merchant_coupon[x] if x in d_merchant_coupon else 1)

all_data['User_Merchant_Used'] = all_data.User_Merchant.apply(lambda x:d_user_merchant_used[x] if x in d_user_merchant_used else 1)
all_data['User_Merchant_Cons'] = all_data.User_Merchant.apply(lambda x:d_user_merchant_cons[x] if x in d_user_merchant_cons else 1)
all_data['User_Merchant_Coupon'] = all_data.User_Merchant.apply(lambda x:d_user_merchant_coupon[x] if x in d_user_merchant_coupon else 1)

In [9]:
all_data.loc[(all_data['Distance'].isnull()), 'Distance'] = 10.

# all_data['Distance'] = all_data.Distance.apply(str)

In [10]:
def count_weekday(x):
    x = str(int(x))
    wd = datetime.datetime(int(x[0:4]), int(x[4:6]), int(x[6:8])).weekday()+1
    return str(wd)

all_data['Weekday'] = all_data.Date_received.apply(count_weekday)

In [11]:
def count_man(x):
    man = int(x.split(':')[0].split('.')[0])
    return man

all_data['Discount_Man'] = all_data.Discount_rate.apply(count_man)

In [12]:
def count_rate(x):
    if '.' in x:
        return float(x)
    elif ':' in x:
        up = int(x.split(':')[1])
        down = int(x.split(':')[0])
        return float(up/down)

all_data['Discount_Newrate'] = all_data.Discount_rate.apply(count_rate)

In [13]:
all_data

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Used,User_Merchant,User_Used,...,User_Coupon,Merchant_Used,Merchant_Cons,Merchant_Coupon,User_Merchant_Used,User_Merchant_Cons,User_Merchant_Coupon,Weekday,Discount_Man,Discount_Newrate
0,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,0.0,"(1439408, 4663)",1,...,4,76,1358,15115,1,1,1,6,150,0.133333
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0.0,"(1439408, 2632)",1,...,4,4,17,39,1,2,3,3,20,0.050000
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0.0,"(1439408, 2632)",1,...,4,4,17,39,1,2,3,6,20,0.050000
3,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN,0.0,"(1439408, 2632)",1,...,4,4,17,39,1,2,3,1,20,0.050000
4,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0,0.0,"(1439408, 2632)",1,...,4,4,17,39,1,2,3,1,20,0.050000
5,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0.0,"(1832624, 3381)",1,...,1,2487,19356,120347,1,1,1,5,200,0.100000
6,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0.0,"(2029232, 3381)",1,...,3,2487,19356,120347,1,1,1,5,200,0.100000
7,2029232,450,1532.0,30:5,0.0,20160530.0,NaN,0.0,"(2029232, 450)",1,...,3,1461,10824,61581,1,1,1,1,30,0.166667
8,2029232,6459,12737.0,20:1,0.0,20160519.0,NaN,0.0,"(2029232, 6459)",1,...,3,1,20,16,1,2,1,4,20,0.050000
9,2747744,6901,1097.0,50:10,10.0,20160606.0,NaN,0.0,"(2747744, 6901)",1,...,1,994,11787,12892,1,1,1,1,50,0.200000


In [14]:
Feature = ['Used', 'User_Used','User_Cons','User_Coupon','Merchant_Used','Merchant_Cons','Merchant_Coupon','User_Merchant_Used','User_Merchant_Cons','User_Merchant_Coupon','Weekday','Discount_Man','Discount_Newrate','Distance']
all_data = all_data[Feature]

In [15]:
all_data = pd.get_dummies(all_data)
print(list(all_data.columns))

['Used', 'User_Used', 'User_Cons', 'User_Coupon', 'Merchant_Used', 'Merchant_Cons', 'Merchant_Coupon', 'User_Merchant_Used', 'User_Merchant_Cons', 'User_Merchant_Coupon', 'Discount_Man', 'Discount_Newrate', 'Distance', 'Weekday_1', 'Weekday_2', 'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6', 'Weekday_7']


In [16]:
all_data.to_csv('part2_all_data.csv',index=False)